In [ ]:
# plt.show()で可視化されない人はこのセルを実行してください。
%matplotlib inline

In [ ]:
# 初めの一回だけこのセルを実行してください、データセットをダウンロードして展開します
# 一回実行すれば、データセットはダウンロードされたままなので、再起動後等再び実行する必要はありません
import urllib.request
import zipfile

# URLを指定
url = "https://storage.googleapis.com/tutor-contents-dataset/4050_cleansing_data.zip"
save_name = url.split('/')[-1]

# ダウンロードする
mem = urllib.request.urlopen(url).read()

# ファイルへ保存
with open(save_name, mode='wb') as f:
    f.write(mem)

# zipファイルをカレントディレクトリに展開する
zfile = zipfile.ZipFile(save_name)
zfile.extractall('.')

# データセットを取得したい時は以下を実行

In [ ]:
!python3 ../distribute_tools/dl_fortutor.py || python ../distribute_tools/dl_fortutor.py

#  OpenCVの利用と画像データの前処理

- **[3.1 画像データの基礎](#3.1-画像データの基礎)**
    - **[3.1.1 RGBデータ](#3.1.1-RGBデータ)**
    - **[3.1.2 画像データのフォーマット](#3.1.2-画像データのフォーマット)**
    - **[3.1.3 透過データ](#3.1.3-透過データ)**
<br><br>
- **[3.2 OpenCVの基礎](#3.2-OpenCVの基礎)**
    - **[3.2.1 画像の読み込み・表示](#3.2.1-画像の読み込み・表示)**
    - **[3.2.2 画像の作成と保存](#3.2.2-画像の作成と保存)**
    - **[3.2.3 トリミングとリサイズ](#3.2.3-トリミングとリサイズ)**
    - **[3.2.4 回転・反転](#3.2.4-回転・反転)**
    - **[3.2.5 色調変換・色反転](#3.2.5-色調変換・色反転)**
<br><br>
- **[3.3 OpenCVの利用](#3.3-OpenCVの利用)**
    - **[3.3.1 閾値処理（二値化）](#3.3.1-閾値処理（二値化）)**
    - **[3.3.2 マスキング](#3.3.2-マスキング)**
    - **[3.3.3 ぼかし（平滑化）](#3.3.3-ぼかし（平滑化）)**
    - **[3.3.4 ノイズの除去](#3.3.4-ノイズの除去)**
    - **[3.3.5 膨張・収縮](#3.3.5-膨張・収縮)**
<br><br>
- **[3.4 まとめ問題（提出不要）](#3.4-まとめ問題（提出不要）)**

***

## <font color="Red"><重要>本講座を開始する前にこちらのコードを実行してください。</b>

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
%matplotlib inline

def aidemy_imshow(name, img):
    if img.ndim == 3:
        b,g,r = cv2.split(img)
        img = cv2.merge([r,g,b])

    plt.gray()
    plt.imshow(img)
    plt.show()

cv2.imshow = aidemy_imshow

## 3.1 画像データの基礎

### 3.1.1 RGBデータ

コンピュータで表示する色は、たった3色の組み合わせでおおよそすべての色を表現しています。これを「光の三原色」と言い、赤（Red）緑（Green）青（Blue）の3色です。余談になりますが、プリンターは基本的にはCMYK、つまりシアン、マゼンダ、イエロー、ブラックで色を表現しています。パソコンのディスプレイは光の発光を利用して色を表現しますが、印刷物は光の吸収を利用して色を表現します。このように、ディスプレイと紙では発色の原理がまったく異なります。<br>
<br>
さて、コンピュータは画像を点の集まりとして扱います。この点は <b>ピクセル</b> と呼ばれ、例えば縦が100ピクセル、横が100ピクセルの画像は100 x 100 = 10,000個のピクセルの集まりという事です。各ピクセルは色の情報を保存する必要があります。例えば、カラー画像であればRGBの3色がそれぞれどのくらいの明るさ(濃さ)であるかを持つ必要があり、多くの場合0から255の範囲で色の明るさを表現します。数字が大きいほど明るくなり、例えば、ただの赤は(255, 0, 0)と表現し、紫色は(255, 0, 255)です。(0, 0, 0)は黒を、(255, 255, 255)は白を表します。<br>
<br>
カラー画像は各ピクセルに3色すべての情報を保存しなければならないので画像のサイズが非常に大きくなります。モノクロ画像は黒1色分の明るさ(0から255)のみを保存すれば良いのでカラー画像に比べてデータ量は3分の1程度で済みます。<br>
後に扱うOpenCVで、1つのピクセルを表すための色要素の数を <b>チャンネル数</b> と呼びます。例えばRGBの画像はチャンネル数が3、モノクロ画像はチャンネル数が1です。

#### 問題

- RGBデータではある色の明るさについての情報が与えられます。どの色の情報が与えられているでしょうか。

1. 赤 
1. 緑
1. 青
1. 上記のすべて

#### ヒント

- RGBが何を意味しているのかを考えます。

#### 解答

上記のすべて

***

### 3.1.2 画像データのフォーマット

画像にはいくつかの種類（画像フォーマット）があります。代表的なものを以下に紹介します。
<br>
<img src="https://aidemyexstorage.blob.core.windows.net/aidemycontents/152974608457650.png">
<br>
前エクササイズで説明した通り、画像は点の集合であり、それぞれの点がRed, Green, Blueの色情報を持っているので、画像のサイズが非常に大きくなります。上の表の右端のBMPは無圧縮、つまり色の情報をそのまま保持します。そのため赤一色の画像（緑と青の色情報がない）であっても緑と青の色情報を保存する領域を確保するため、サイズが非常に大きくなります。
BMP以外は色情報を効率的に圧縮する工夫をしています。JPGは写真を扱う事が得意で元の写真を大きく圧縮できますが、圧縮後の写真を元の写真に戻せなくなってしまう特徴があり、これを非可逆圧縮と言います。それに対してPNGは写真の圧縮率はJPGに劣るものの、可逆圧縮であり、圧縮した画像を元のクオリティに完全に戻す事ができます。GIFは古いフォーマットなので色数や圧縮率では劣りますが、簡単なアニメーションを表現する事ができます。

#### 問題

- アニメーションの表現に適している画像フォーマットを選択してください。

1. PNG 
1. JPG
1. BMP
1. GIF

#### ヒント

- 表現できる色数が少ないフォーマットです。

#### 解答

GIF

***

### 3.1.3 透過データ

画像の背景を透明にすることを透過といいます。透過させる方法は、ソフトを使ったり、画像の作成段階で透過させたり色々な方法があります。なお、後に紹介するOpenCVでも可能ですが、本コースでは扱いません。<br>
<br>
非透過PNG
<img src="https://aidemyexstorage.blob.core.windows.net/aidemycontents/1529748720992822.png" alt="非透過">
<br>
<br>
透過PNG
<img src="https://aidemyexstorage.blob.core.windows.net/aidemycontents/1529750614672625.png" alt="透過">
<br>
透過処理をサポートしている画像フォーマットはGIFとPNGですが、表示するプログラムがどのように色を扱うかによっても見え方は異なります。例えば、BMPは透過処理をサポートしていませんが、BMPで作られているアイコン画像が透過されているように見える場合があります。これはアイコンを表示するプログラムが特定の位置の色を透過色として扱うためです。

#### 問題

- 透過処理がサポートされていない画像フォーマットを選択してください。

1. JPG
1. GIF
1. PNG
1. 上記の全て

#### ヒント

- 比較的画像サイズが小さいフォーマットです。

#### 解答

JPG

***

## 3.2 OpenCVの基礎

### 3.2.1 画像の読み込み・表示

OpenCVは、画像を扱うのに便利なライブラリです。このセクションではcv2を用いた基本的な画像の取り扱いについて学びます。<br>
<br>
まずは画像を読み込んで、出力します。<br>
画像を読み込むにはOpenCVの<b style='color: #AA0000'>cv2.imread()</b>関数を用います。
```Python
cv2.imread("ファイル名")
```
画像を出力するには<b style='color: #AA0000'>cv2.imshow()</b>関数を用います。画像を表示するウィンドウのサイズは自動で調整されます。
```Python
cv2.imshow("ウィンドウ名", 読み込んだ画像データ)
```
***
画像を読み込んで、出力してみます。
```Python
# ライブラリをインポートします
import numpy as np
import cv2

# 画像を読み込みます
img = cv2.imread("./4050_data_cleansing_data/sample.jpg")

# 画像を表示するウィンドウの名前は"sample"
# 読み込んだ画像は"img"です
cv2.imshow("sample", img)
```

#### 問題

- フォルダ「4050_data_cleansing_data」内の画像`sample.jpg`を出力してください。
- ウィンドウ名は`sample`としてください。

In [ ]:
import numpy as np
import cv2

# OpenCVを用いて画像を読み込んでください


# 画像を出力してください


#### ヒント

- `cv2.imread("ファイル名")`で画像を読み込むことができます。
- `cv2.imshow("ウィンドウ名", 画像データ)`で画像を出力することができます。

#### 解答例

In [ ]:
import numpy as np
import cv2

# OpenCVを用いて画像を読み込んでください
img = cv2.imread("./4050_data_cleansing_data/sample.jpg")

# 画像を出力してください
cv2.imshow("sample", img)

In [ ]:
index_code_contains("imread", failure_msg="imreadを用いて画像を読み込んでください。")
index_code_contains("data\/sample\.jpg", failure_msg="4050_cleansing_data/sample.jpgを読み込んでください。")
index_code_contains("imshow", failure_msg="imshowを用いて画像を表示してください。")

***

### 3.2.2 画像の作成と保存

単色画像の作成と保存の方法です。cv2で画像を処理する際は色の指定の順番はRGBではなく、BGRになるので注意が必要です。 <br>
<br>
画像の作成にはNumPyの行列を作成する<b style='color: #AA0000'>np.array()</b>関数を用い、さらに`for`文と`range()`関数を組み合わせた多重ループにより各ピクセルの色情報を並べて描画するような処理を行います。下のサンプルコードは、`[B, G, R]` の値を指定し、1つめのforで横に512枚の画像、2つめのforで縦に512枚の画像を作成する多重ループとなっています。<br>
データ型には、8ビット符号なしの整数型で値の範囲が0～255と定められている`uint8`を指定します。B, G, Rの色情報は<a href='/courses/4050/exercises/BJtvth8jIlM' target='_blank'>3.1.1 RGBデータ</a>で学んだ通り、0〜255の範囲で表現されるためです。
```Python
np.array([[[B, G, R] for _ in range(横の画像サイズ)] for _ in range(縦の画像サイズ)], dtype="uint8")
```
「`for _ in range`」の \_ は、for文で処理を繰り返す際に、\_ にあたる変数名を明示しない場合に使用します。<br>
<br>
画像を保存するにはOpenCVの<b style='color: #AA0000'>cv2.imwrite()</b>関数を用います。
```Python
cv2.imwrite("ファイル名", 作成した画像データ)
```
***
例えば、画像サイズが縦512px、横512pxの赤一色の画像を作成します。
```Python
import numpy as np
import cv2

# 画像のサイズを決めます
img_size = (512, 512)

# 画像の情報を持つ行列をNumPyで作ります
# 赤色の画像なので各要素が[0, 0, 255]の、512 * 512の行列を作ることを考えます
# 行列が転置されることに注意します
# 画像データの各要素は、0~255の値しか取らないので、このことを明示するためにdtypeオプションでデータ型を定めます
my_img = np.array([[[0, 0, 255] for _ in range(img_size[1])] for _ in range(img_size[0])], dtype="uint8")

# 表示
cv2.imshow("sample", my_img)

# 保存
# ファイル名は"my_red_img.jpg"
cv2.imwrite("my_red_img.jpg", my_img)
```

#### 問題

- 画像サイズが512 $\times$ 512ピクセルの緑色の画像を作成し、表示してください。<br>
緑色のRGBは[0, 255, 0]です。
- 作成した画像をファイル名"my_green_img.jpg"で保存してください。

In [ ]:
import numpy as np
import cv2

# 画像のサイズを決める
img_size = 

# サイズが512 × 512の緑色の画像を作ってください


# 作成した画像を表示してください


# 画像をファイル名"my_green_img.jpg"で保存してください


#### ヒント

- for文を用い、各要素が`[0, 255, 0]`の512$\times$512のnp.arrayを作ります。
- `cv2.imshow("ウィンドウ名", 画像データ)`で画像を出力することができます。
- `cv2.imwrite("ファイル名", 画像データ)`でファイルを保存することができます。

#### 解答例

In [ ]:
import numpy as np
import cv2

# 画像のサイズを決める
img_size = (512, 512)

# サイズが512 × 512の緑色の画像を作ってください
img = np.array([[[0, 255, 0] for _ in range(img_size[1])] for _ in range(img_size[0])], dtype="uint8")

# 作成した画像を表示してください
cv2.imshow("sample", img)

# 画像をファイル名"my_green_img.jpg"で保存してください
cv2.imwrite("my_green_img.jpg", img)

In [ ]:
index_code_contains("\[0,255,0", failure_msg="緑色の画像を作成してください。")
index_code_contains("imshow", failure_msg="imshowを用いて画像を表示してください。")
index_code_contains("imwrite", failure_msg="imwriteを用いて画像を保存してください。")
index_code_contains("my\_green\_img\.jpg", failure_msg="ファイル名はmy_green_img.jpgです。")

***

### 3.2.3 トリミングとリサイズ

画像のトリミングとリサイズの方法です。トリミングとは画像の一部を取り出す操作のことで、リサイズとは画像のサイズを変える（拡大、縮小）することです。<br>
<br>
トリミングは画像を表す行列を範囲指定することで、画像の指定した部分を四角く切り取ります。<br>
範囲指定をする際、画像の左上が`0`です。

```Python
img[y始点：y終点, x始点：x終点]
```

リサイズは<b style='color: #AA0000'>cv2.resize()</b>関数を用います。

```Python
cv2.resize(画像データ, (リサイズ後の幅, リサイズ後の高さ))
```

***

```Python
import numpy as np
import cv2
import matplotlib.pyplot as plt

# 画像を読み込む
# cv2.imread は画像をNumPyのndarray型で読み込む
img = cv2.imread("./4050_data_cleansing_data/sample.jpg")

# 読み込んだ画像データは（高さ　x 幅 x 色数)の3重配列
size = img.shape # サイズ (1000, 667, 3)

# 範囲を指定して取り出す（範囲指定は固定値もスライスも可能）
# n等分したい時はサイズの商を取るが、小数点以下は切り捨てる
my_img = img[: size[0] // 2, : size[1] // 3]

# 新たにサイズを指定する際、(幅, 高さ)の順になる
# 元の倍率を保ったまま、幅と高さをそれぞれ2倍する場合
my_img = cv2.resize(my_img, (my_img.shape[1] * 2, my_img.shape[0] * 2))

# 表示
# トリミングおよびリサイズで指定した範囲が分かりやすいように目盛りが表示されるMatplotlib を用いて表示
my_img = cv2.cvtColor(my_img, cv2.COLOR_BGR2RGB)
plt.imshow(my_img)
plt.show()

# cv2.imshowで表示する場合
# cv2.imshow("sample", my_img)
```

***
リサイズの応用として、画像を縮小して画素数を落としてから元のサイズに戻すと、画像にモザイクをかけることができます。

#### 問題

- サンプルコードで使用した`sample.jpg`の幅、高さをそれぞれ1/3にリサイズしてください。

In [ ]:
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg")
print(img.shape)

# 幅、高さをそれぞれ1/3にリサイズします
my_img = 

# 出力
cv2.imshow("sample", my_img)
print(my_img.shape)

#### ヒント

- 画像のリサイズは`cv2.resize(画像データ, (リサイズ後の幅, リサイズ後の高さ))`を用います。
- img.shapeで画像の行(高さ)、列(幅)を取得しその値を1/3します。

#### 解答例

In [ ]:
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg")
print(img.shape)

# 幅、高さをそれぞれ1/3にリサイズします
my_img = cv2.resize(img, (img.shape[1] // 3, img.shape[0] // 3))

# 出力
cv2.imshow("sample", my_img)
print(my_img.shape)

In [ ]:
index_code_contains("imread", failure_msg="imreadを用いて画像を読み込んでください。")
index_code_contains("sample.jpg", failure_msg="sample.jpgを読み込んでください。")
index_code_contains("resize", failure_msg="resizeを用いてリサイズをしてください。")
index_code_contains("0\](?:\/)?\/3", failure_msg="1/3にリサイズしてください。")
index_code_contains("1\](?:\/)?\/3", failure_msg="1/3にリサイズしてください。")
index_code_contains("imshow", failure_msg="imshowを用いて画像を表示してください。")

***

### 3.2.4 回転・反転

画像を回転するには<b style='color:#AA0000'>cv2.warpAffine()</b>関数を用います。この関数は<b>アフィン変換</b>と呼ばれる幾何変換を行います。<br>
アフィン変換は平行移動と線形変換（拡大・縮小、剪断（スキュー）、回転）の組み合わせで表現されます。

```Python
cv2.warpAffine(元の画像データ, 変換行列, 出力画像のサイズ)
```

パラメータは以下の通りです。
- 第一引数：回転する元画像データ（NumPy配列ndarray）<br>
- 第二引数：変換行列（NumPy配列ndarray）<br>
- 第三引数：出力画像のサイズ（タプル）

第二引数に指定する変換行列は<b style='color:#AA0000'>cv2.getRotationMatrix2D()</b>関数で取得します。

```Python
cv2.getRotationMatrix2D(画像の中心の座標,　回転させたい角度,　拡縮倍率)
```

パラメータは以下の通りです。
- 第一引数：画像の中心の座標（タプル）<br>
- 第二引数：回転させたい角度（ラジアンではありません）<br>
- 第三引数：拡大・縮小の倍率（等倍が`1.0`です）

```Python
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg") # サイズ (1000, 667, 3)

# warpAffine()関数の第二引数である`変換行列`を取得します
mat = cv2.getRotationMatrix2D(tuple(np.array([img.shape[1], img.shape[0]]) / 2), 180, 2.0)

# 倍率2倍で180度回転します
my_img = cv2.warpAffine(img, mat, img.shape[::-1][1:3])

cv2.imshow("sample", my_img)
```

***
反転は<b style='color:#AA0000'>cv2.flip()</b>関数を用います。対象とする軸が0のときx軸中心に（上下反転）、正のときy軸中心に（左右反転）、負のとき両方の軸を中心に反転します。

```Python
cv2.flip(画像データ, 軸)
```

パラメータは以下の通りです。
- 第一引数：反転する元画像データ<br>
- 第二引数：対象の軸

```Python
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg")

# x軸y軸両方の軸を中心に反転します
my_img = cv2.flip(img, -1)

cv2.imshow("sample", my_img)
```

#### 問題

- 画像をx軸を中心に反転してください。 

In [ ]:
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg")

# x軸を中心に反転してください


cv2.imshow("sample", my_img)

#### ヒント

- `cv2.flip()`は引数が0のときx軸中心に、正のときy軸中心に、負のとき両方の軸を中心に反転します。

#### 解答例

In [ ]:
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg")

# x軸を中心に反転してください
my_img = cv2.flip(img, 0)

cv2.imshow("sample", my_img)

In [ ]:
index_code_contains("imread", failure_msg="imreadを用いて画像を読み込んでください。")
index_code_contains("sample.jpg", failure_msg="sample.jpgを読み込んでください。")
index_code_contains("flip", failure_msg="flipを用いて画像を反転してください。")
index_code_contains("0", failure_msg="x軸を中心に反転してください。")
index_code_contains("imshow", failure_msg="imshowを用いて画像を表示してください。")

***

### 3.2.5 色調変換・色反転

画像を他の色空間に変換します。OpenCVは色々な色空間をサポートしており、<b style='color: #AA0000'>cv2.cvtColor()</b>関数を用いて相互に変換することが可能です。
```Python
cv2.cvtColor(画像データ, 他の色空間に変換するcode)
```
例えば、BGR色空間の画像をLab色空間に変換します。Lab色空間は、人間の視覚に近似するよう設計されている点が優れている表色系です。
```Python
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg")

# BGRをLab色空間に変換
my_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

cv2.imshow("BGR to LAB", my_img)
```
cv2.imreadはデフォルトでBGR色空間で画像を読み込みますが、例えば、MatplotlibやSciPyのデフォルトはRGBです。<a href='/courses/4050/exercises/HyCPt28o8lz' target='_blank'>3.2.3 トリミングとリサイズ</a>の説明内で目盛りを表示するためにMatplotlibで画像を表示していますが、その際には`COLOR_BGR2RGB`を用いました。また、`COLOR_BGR2GRAY`を用いると、グレースケールに変換することができます。<br>
その他の変換コードは<a href='https://docs.opencv.org/3.4.0/d7/d1b/group__imgproc__misc.html#ga4e0972be5de079fed4e3a10e24ef5ef0' target='_blank'>OpenCVの公式サイト</a>をご参照ください。
***
画像の色を反転させることをネガポジ反転といいます。モノクロ画像の場合は白黒反転し、フルカラー画像の場合は補色反転を行います。<br>
RGBの値は0〜255で構成されているので、ある画素値xを255-xに置き換えることで色を反転できます。OpenCVで読み込んだ画素値は3次元のnumpy配列になっています。その配列を１画素ずつRGBチャネルごとにxから255-xに書き換えます。
```Python
# 各画素の座標(i, j)とRGBチャネル(k)を指定して変換前の画素値を取得
img[i][j][k] = x
```
for文と、len(img[i])で座標を取得して各画素に順番にアクセスし、RGBチャネルごとに画素値を変換していきます。
```Python
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg")
# img.shapeは(1000, 667, 3)
# それぞれが i, j, k の値となる

# 画素の座標(i, j)を取得
for i in range(len(img)):
    for j in range(len(img[i])):
        # RGBチャネル(k:0-2)を取得
        for k in range(len(img[i][j])):
            # 座標(i, j)のRGBチャネル(k:0-2)ごとに画素値を変換
            img[i][j][k] = 255 - img[i][j][k]

cv2.imshow("sample", img)
```
OpenCVでは<b style='color: #AA0000'>cv2.bitwise_not()</b>関数を用いて反転します。
```Python
img = cv2.bitwise_not(画像データ)
```
`cv2.bitwise()`関数は8bitで表わされている各画素のビットを操作することができます。`not`で各ビットを反転させます。

#### 問題

- `sample.jpg`の色をOpenCVの関数を用いて色反転してください。

In [ ]:
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg")

# OpenCVで色を反転します


cv2.imshow("sample", my_img)

#### ヒント

- 色を反転するには`cv2.bitwise_not()`関数を用います。

#### 解答例

In [ ]:
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg")

# OpenCVで色を反転します
my_img = cv2.bitwise_not(img)

cv2.imshow("sample", my_img)

In [ ]:
index_code_contains("imread", failure_msg="imreadを用いて画像を読み込んでください。")
index_code_contains("sample\.jpg", failure_msg="sample.jpgを読み込んでください。")
index_code_contains("bitwise\_not", failure_msg="bitwise_notを用いて画像を読み込んでください。")
index_code_contains("imshow", failure_msg="imshowを用いて画像を表示してください。")

***

## 3.3 OpenCVの利用

### 3.3.1 閾値処理（二値化）

画像の容量を小さくするために、一定以上明るい色、あるいは一定以上暗い色をすべて同じ値にする、要は白と黒に二値化する処理を閾値（しきいち）処理と呼びます。閾値処理は<b style='color: #AA0000'>cv2.threshold()</b>関数を用います。
```Python
cv2.threshold(画像データ, 閾値, 最大値 maxValue, 閾値処理の種類)
```
パラメータは以下の通りです。
- 第一引数：二値化する元画像ファイル名
- 第二引数：閾値とする画素値
- 第三引数：閾値に対する最大濃度の値
- 第四引数：OpenCVの閾値処理の種類（THRESH_BINARY, THRESH_BINARY_INV, THRESH_TRUNC, THRESH_TOZERO, THRESH_TOZERO_INVのいずれか）

第四引数の閾値処理の種類は以下の通りです。
- THRESH_BINARY　  ： 閾値を超えるピクセルは maxValue に、それ以外のピクセルは 0 になります
- THRESH_BINARY_INV： 閾値を超えるピクセルは 0 に、それ以外のピクセルは maxValue になります
- THRESH_TRUNC　   ： 閾値を超えるピクセルは 閾値 に、それ以外のピクセルは変更されません
- THRESH_TOZERO　  ： 閾値を超えるピクセルは変更されず、それ以外のピクセルは 0 になります
- THRESH_TOZERO_INV： 閾値を超えるピクセルは 0 に、それ以外のピクセルは変更されません

閾値処理の種類により様々な閾値処理を行うことができます。
```Python
import numpy as np
import cv2

# カラー画像をグレースケールで読み込む（cv2.IMREAD_GRAYSCALE または 0）
img = cv2.imread("./4050_data_cleansing_data/sample.jpg", 0)

# THRESH_TOZEROを使い、閾値を150、最大値を255 に設定します
# 閾値も返されるので retval で受け取ります
retval, my_img = cv2.threshold(img, 150, 255, cv2.THRESH_TOZERO)

cv2.imshow("original", img)
cv2.imshow("sample", my_img)
print(retval)
```

#### 問題

- 閾値を100にし、それ以下を0, それ以上を255にして二値化して出力してください。
- 閾値の返り値も取得してprint文で出力してください。

In [ ]:
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg", 0)

# 閾値100、それ以下を0、それ以上を255にして二値化してください
# 同時に閾値も取得してください


# 出力



#### ヒント

- 閾値より大きい値はmaxValueに、それ以外は0になる`THRESH_BINARY`を用います。

#### 解答例

In [ ]:
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg", 0)

# 閾値100、それ以下を0、それ以上を255にして二値化してください
# 同時に閾値も取得してください
retval, my_img = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)

# 出力
cv2.imshow("sample", my_img)
print(retval)

In [ ]:
index_code_contains("imread", failure_msg="imreadを用いて画像を読み込んでください。")
index_code_contains("sample.jpg", failure_msg="sample.jpgを読み込んでください。")
index_code_contains("threshold", failure_msg="thresholdを用いて閾値処理を行ってください。")
index_code_contains("THRESH_BINARY", failure_msg="THRESH_BINARYを用いてください。")
index_code_contains("img, 100, 255, cv2.THRESH_BINARY", failure_msg="引数の数値が異なります。")
index_code_contains("imshow", failure_msg="imshowを用いて画像を表示してください。")
index_code_contains("print", failure_msg="printを用いて閾値を出力してください。")

### 3.3.2 マスキング

元画像の一部分を別の画像の白で描画された範囲でくり抜いたように表示します。マスキング処理には、白黒のチャンネル数が`1`のマスク用の画像を用意します。マスク用画像の黒い部分で元画像をマスクし、白い部分のみを表示します。マスキング処理は<b style='color: #AA0000'>cv2.bitwise_and()</b>関数を用います。
```Python
cv2.bitwise_and(元画像データ1, 元画像データ2(元画像データ1と同じで可), mask=マスク用画像)
```
```Python
import numpy as np
import cv2

# 元画像を読み込みます
img = cv2.imread("./4050_data_cleansing_data/sample.jpg")

# マスク用画像を、第二引数に0を指定してチャンネル数が1の画像（グレースケール画像）に変換して読み込みます
mask = cv2.imread("./4050_cleansing_data/mask.png", 0)

# マスク用画像を、元画像と同じサイズにリサイズします
mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

# 第三引数にマスク用画像を指定してマスキングします
my_img = cv2.bitwise_and(img, img, mask = mask)

cv2.imshow("sample", my_img)
```
***
上のサンプルコードで使用しているマスク用画像（mask.png）です。
<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/4050_data_cleansing/cleansing_chap3_20.png" width="100px">

#### 問題

- sample.jpgを、mask.pngの黒い部分のみを表示するようマスキングしてください 。

In [ ]:
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg")
mask = cv2.imread("./4050_data_cleansing_data/mask.png", 0)
mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

# mask.pngの黒い部分のみを表示します（黒の画素数は`0`です）
retval, mask = 
my_img = 

# 出力
cv2.imshow("sample", my_img)

#### ヒント

- マスク用画像の黒い部分を表示することはできないので、`cv2.threshold()`を用いて白黒を反転します。
- 白と黒を反転するには`cv2.THRESH_BINARY_INV`を用います。
- マスキングには`bitwise_and()`を用います。

#### 解答例

In [ ]:
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg")
mask = cv2.imread("./4050_data_cleansing_data/mask.png", 0)
mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

# mask.pngの黒い部分のみを表示します（黒の画素数は`0`です）
retval, mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY_INV)
my_img = cv2.bitwise_and(img, img, mask = mask)

# 出力
cv2.imshow("sample", my_img)

In [ ]:
index_code_contains("imread", failure_msg="imreadを用いて画像を読み込んでください。")
index_code_contains("sample.jpg", failure_msg="sample.jpgを読み込んでください。")
index_code_contains("threshold", failure_msg="thresholdを用いてマスク用画像を反転してください。")
index_code_contains("mask, 0, 255, cv2.THRESH_BINARY_INV", failure_msg="閾値処理の数値が異なります。")
index_code_contains("bitwise_and", failure_msg="bitwise_andを用いてマスク処理をしてください。")
index_code_contains("imshow", failure_msg="imshowを用いて画像を表示してください。")

### 3.3.3 ぼかし（平滑化）

画像をぼかすにはいくつかの方法がありますが、その中でもあるピクセル（画素）の周りのn✕n個のピクセルを平均化させてぼかす、ガウシアンフィルタを用いて画像全体をぼかします。ぼかし処理は<b style='color: #AA0000'>cv2.GaussianBlur()</b>関数を用います。
```Python
cv2.GaussianBlur(画像データ, カーネルサイズ(n, n), 0)
```
パラメータは以下の通りです。
- 第一引数：ぼかし処理を行う元画像ファイル名
- 第二引数：カーネルサイズ（n ✕ n の値（nは奇数））
- 第三引数：ガウシアンカーネルのx方向の標準偏差（通常は0）

カーネルサイズと標準偏差の値を大きくするほど、ぼかしが強くなります。
```Python
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg")

my_img = cv2.GaussianBlur(img, (51, 51), 0)

cv2.imshow("sample", my_img)```

#### 問題

- 画像にぼかしを施してください。

In [ ]:
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg")

# sample.jpg をぼかしてください
my_img = 

cv2.imshow("sample", my_img)

#### ヒント

- 画像をぼかすには`cv2.GaussianBlur()` 関数を用います。

#### 解答例

In [ ]:
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg")

# sample.jpg をぼかしてください
my_img = cv2.GaussianBlur(img, (101, 101), 0)

cv2.imshow("sample", my_img)

In [ ]:
index_code_contains("imread", failure_msg="imreadを用いて画像を読み込んでください。")
index_code_contains("sample.jpg", failure_msg="sample.jpgを読み込んでください。")
index_code_contains("GaussianBlur", failure_msg="GaussianBlurを用いてぼかし処理をしてください。")
index_code_contains("imshow", failure_msg="imshowを用いて画像を表示してください。")

***

### 3.3.4 ノイズの除去

カラー画像のノイズを除去するには<b style='color: #AA0000'>cv2.fastNlMeansDenoisingColored()</b>関数を用い、グレースケール画像のノイズ除去には<b style='color: #AA0000'>cv2.fastNlMeansDenoising()</b>関数を用います。
```Python
# カラー画像
cv2.fastNlMeansDenoisingColored(画像データ)
# グレースケール画像
cv2.fastNlMeansDenoising(画像データ)
```
```Python
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg")

my_img = cv2.fastNlMeansDenoisingColored(img)

cv2.imshow("sample", my_img)
```
出力結果
<img src="https://aidemyexstorage.blob.core.windows.net/aidemycontents/1555066229082593.jpg" alt="ノイズ出力結果">
拡大してみると、特に顔にかかっているノイズや白のシャツのノイズが除去されていることが分かりやすく確認できます。

#### 問題

- カラー画像のノイズを除去する関数はどれでしょうか。

- `fastNlMeansNoisingColored`
- `fastNlMeansDenoisingColored`
- `fastNlMeansNoisingDeColored`
- `fastNlMeansDenoisingDeColored`

#### ヒント

- ノイズ処理は`Denoising`です。

#### 解答

- `fastNlMeansDenoisingColored`

***

### 3.3.5 膨張・収縮

<a href='/courses/4050/exercises/r14dt38iLlz' target='_blank'>3.3.4 ノイズの除去</a>ではノンローカルミーンフィルタ（Non-local Means Filter）によりカラー画像とグレースケール画像のノイズ除去を行いましたが、膨張・収縮処理によってもノイズを取り除くことが可能です。膨張と収縮は主に二値画像で用いられることが多い手法で、膨張はあるピクセルを中心とし、フィルタ内の最大値（=白）をその中心の値にします。逆に収縮は最小値（=黒）をその中心の値とします。フィルタは、中心のピクセルの上下左右4つを用いる方法と、自身を囲む8つを用いる方法の2通りが主です。<br>
膨張は<b style='color: #AA0000'>cv2.dilate()</b>関数を、収縮は<b style='color: #AA0000'>cv2.erode()</b>関数を用います。
```Python
# 膨張
cv2.dilate(画像データ, フィルタ)
# 収縮
cv2.erode(画像データ, フィルタ)
```
パラメータは以下の通りです。
- 第一引数：元画像ファイル名
- 第二引数：フィルタ（カーネルサイズ）

フィルタの指定はnumpyのndarray型の配列とタプル、どちらでも構いません。
```Python
# グレースケールで読み込み
img = cv2.imread("./4050_data_cleansing_data/sample.jpg", 0)

# 二値画像に変換
retval, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# フィルタの定義
filt = np.array([[0, 1, 0],
                [1, 0, 1],
                [0, 1, 0]], np.uint8)
# filt = np.ones((3, 3), np.uint8) でもOK

# 膨張
my_img = cv2.dilate(img, filt)

cv2.imshow("sample", my_img)
```
`np.uint8`はデータの型を表しており、8ビットで表される符号なしの整数型です。詳しくは<a href='/courses/4050/exercises/rk6vt2LsUgG' target='_blank'>3.2.2 画像の作成と保存</a>で説明しています。<br>
<br>
例えば、ノイズの入った円の画像に対して膨張・収縮処理を行うと以下のようになります。

<img src="https://aidemyexstorage.blob.core.windows.net/aidemycontents/155929367874492.png" alt="収縮,膨張処理例">

#### 問題 

- sample.jpgファイルを説明文と同じフィルタを用いて収縮してください。

In [ ]:
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg", 0)
retval, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# フィルタを定義してください
filt = 



# 収縮してください
my_img = 

# 表示
cv2.imshow("sample", my_img)
# 比較のため元の写真も表示
cv2.imshow("original", img)

#### ヒント

- 画像を収縮するには`cv2.erode()`関数を用います。

#### 解答例

In [ ]:
import numpy as np
import cv2

img = cv2.imread("./4050_data_cleansing_data/sample.jpg", 0)
retval, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# フィルタを定義してください
filt = np.array([[0, 1, 0],
                [1, 0, 1],
                [0, 1, 0]], np.uint8)

# 収縮してください
my_img = cv2.erode(img, filt)

# 表示
cv2.imshow("sample", my_img)
# 比較のため元の写真も表示
cv2.imshow("original", img)

In [ ]:
index_code_contains("imread", failure_msg="imreadを用いて画像を読み込んでください。")
index_code_contains("sample.jpg", failure_msg="sample.jpgを読み込んでください。")
index_code_contains("erode", failure_msg="erodeを用いて収縮してください。")
index_code_contains("\[\[0,1,0], \[1,0,1], \[0,1,0]]", failure_msg="フィルタが異なります。")
index_code_contains("imshow", failure_msg="imshowを用いて画像を表示してください。")

***

## 3.4 まとめ問題（提出不要）

画像処理の問題を解いていただきます。

#### 問題

- 以下のコメントアウトの処理を行ってください

In [ ]:
import cv2
import numpy as np


img = cv2.imread("./4050_data_cleansing_data/sample.jpg")
# オリジナル
cv2.imshow('Original', img)

# ぼかし処理を作成して下さい。(第二引数は77, 77を指定してください。)
blur_img = 
cv2.imshow('Blur', blur_img)

# 画像の色を反転させてください。
bit_img = 
cv2.imshow('Bit', bit_img)

# 閾値処理をしてください。(閾値を90にし、それ以下を変更なし, それ以上を０にしてください。)
retval, thre_img = 
cv2.imshow('THRESH', thre_img)

#### ヒント

このチャプターで学んだことを復習しましょう。

#### 解答例

In [ ]:
import cv2
import numpy as np


img = cv2.imread("./4050_data_cleansing_data/sample.jpg")
# オリジナル
cv2.imshow('Original', img)

# ぼかし処理を作成して下さい。(第二引数は77, 77を指定してください。)
blur_img = cv2.GaussianBlur(img, (77, 77), 0)
cv2.imshow('Blur', blur_img)

# 画像の色を反転させてください。
bit_img = cv2.bitwise_not(img)
cv2.imshow('Bit', bit_img)

# 閾値処理をしてください。(閾値を90にし、それ以下を変更なし, それ以上を０にしてください。)
retval, thre_img = cv2.threshold(img, 90, 255, cv2.THRESH_TOZERO_INV)
cv2.imshow('THRESH', thre_img)


***